## Combining Data Sources
This notebook contains several steps:
1. collect features from sat images
2. combine those with weather features and elevation features 

In [29]:
import cv2
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

import os

In [30]:
# get path to the data folder
DATA_DIR = Path.cwd().parent.resolve() / "data"
assert DATA_DIR.exists()

## 1. Extract features from satellite image files

In [31]:
def get_image_features(image_layers):
    # count water pixels
    num_water_pxl = (image_layers[4] > 0.0).sum()
    # calculate mean color values from masked (water-only) layers
    mean_r  = image_layers[4].sum()/(num_water_pxl) if (num_water_pxl > 0) else 0.0
    mean_g  = image_layers[5].sum()/(num_water_pxl) if (num_water_pxl > 0) else 0.0
    mean_b  = image_layers[6].sum()/(num_water_pxl) if (num_water_pxl > 0) else 0.0
    mean_ir = image_layers[7].sum()/(num_water_pxl) if (num_water_pxl > 0) else 0.0
    
    # calculate HSV transformed image from water-only layers
    rgb = np.array([image_layers[4],image_layers[5],image_layers[6]])
    hsv = np.transpose(cv2.cvtColor(np.transpose((rgb), axes=[1, 2, 0]), cv2.COLOR_RGB2HSV), axes=[2, 0, 1])

    # calculate mean values for hue, saturation and value (hue is scaled from 360° into 0...1 range)
    mean_h  = hsv[0].sum()/(num_water_pxl*360) if (num_water_pxl > 0) else 0.0
    mean_s  = hsv[1].sum()/(num_water_pxl) if (num_water_pxl > 0) else 0.0
    mean_v  = hsv[2].sum()/(num_water_pxl) if (num_water_pxl > 0) else 0.0

    features = {'lake_size': num_water_pxl,
                'mean_r': mean_r, 
                'mean_g': mean_g, 
                'mean_b': mean_b, 
                'mean_ir': mean_ir, 
                'mean_h': mean_h,
                'mean_s': mean_s,
                'mean_v': mean_v}

    return features

In [32]:
# df = pd.read_csv(DATA_DIR / "dataset_long_sat_images.csv", parse_dates=["date"])
# df2 = pd.read_csv(DATA_DIR / "dataset_sat_images.csv", parse_dates=["date"])
# df = df.merge(df2[['uid','image_src']] , how="outer", on='uid', validate="1:1")




# df['image_src'] = 
# df.head(20)

In [33]:
df = pd.read_csv(DATA_DIR / "dataset_long_sat_images.csv", parse_dates=["date"])

# remove id's where no sat images could be collected
print("Initial entries in dataset: ", len(df))
#errored_ids = ['einx', 'gygq', 'ifwc', 'jdvp', 'qpeh', 'tgiq', 'wrqa']
errored_ids = ['alwh', 'bslg', 'dbci', 'egjs', 'eqkm', 'gwei', 'gygq', 'hdvw', 
                'hhlw', 'hklc', 'hsqr', 'ifel', 'ifwc', 'jdvp', 'jmvw', 'juny',
                'jztb', 'ljln', 'msjt', 'nttf', 'nyul', 'oesl', 'ojsc', 'oomm',
                'prpz', 'qken', 'qpeh', 'rygl', 'ugqe', 'uhfr', 'ushn', 'vblk',
                'wrqa', 'wvkh', 'wzxe', 'einx', 'tgiq']
for id in errored_ids:
    df.drop(df.loc[df['uid']==id].index, inplace=True)
print("Remaining entries in dataset: ", len(df))
image_count = len(df)

Initial entries in dataset:  23570
Remaining entries in dataset:  23533


In [34]:
# add filepaths to dataframe
def get_path(id):
   return str(DATA_DIR/'sat_images/image_arrays_8_layer_2/') + f"/{id}.npy"

df['path'] = df["uid"].apply(get_path)

# pd.set_option('display.max_colwidth', None)
# df.head()

In [35]:
# use only part of the dataset where images were found
#image_df = df[df['image_src'] != 'no_image']

image_df = df

In [37]:
# extract features from all images
for row in tqdm(image_df.itertuples(), total=len(image_df)):
    # load image file
    image_layers = np.float32(np.load(row.path))

    # extract features
    features = get_image_features(image_layers)

    # add features to dataframe
    df.loc[df['uid'] == row.uid, ['lake_size']] = features['lake_size']
    df.loc[df['uid'] == row.uid, ['mean_r']] = features['mean_r']
    df.loc[df['uid'] == row.uid, ['mean_g']] = features['mean_g']
    df.loc[df['uid'] == row.uid, ['mean_b']] = features['mean_b']
    df.loc[df['uid'] == row.uid, ['mean_ir']] = features['mean_ir']
    df.loc[df['uid'] == row.uid, ['mean_h']] = features['mean_h']
    df.loc[df['uid'] == row.uid, ['mean_s']] = features['mean_s']
    df.loc[df['uid'] == row.uid, ['mean_v']] = features['mean_v']

# column filepath is not needed anymore
df = df.drop('path', axis=1)

100%|██████████| 23533/23533 [04:05<00:00, 95.66it/s]


In [46]:
# remove 0 values
df['lake_size'][df['lake_size'] == 0.0] = np.nan
for feat in ['lake_size', 'mean_r','mean_g','mean_b','mean_ir','mean_h','mean_s','mean_v',]:
    df[feat][df['lake_size'].isna()] = np.nan

/var/folders/x6/y83h9zms7bq9mpq4000kg4tc0000gn/T/ipykernel_40870/2388970049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lake_size'][df['lake_size'] == 0.0] = np.nan
/var/folders/x6/y83h9zms7bq9mpq4000kg4tc0000gn/T/ipykernel_40870/2388970049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feat][df['lake_size'].isna()] = np.nan


In [47]:
df.head()

,uid,latitude,longitude,date,split,region,severity,density,image_src,lake_size,mean_r,mean_g,mean_b,mean_ir,mean_h,mean_s,mean_v
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,NaN,2578.0,0.467581,0.607486,0.536248,0.424409,0.413782,0.234153,0.607841
1,aabn,36.559700,-121.510000,2016-08-31,test,NaN,NaN,NaN,no_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,NaN,176.0,0.112471,0.111118,0.057916,0.059888,0.163396,0.503357,0.115065
3,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,NaN,1495.0,0.065787,0.085121,0.044582,0.020998,0.245394,0.480389,0.085137


In [48]:
# save image features to csv file
df.to_csv(DATA_DIR / "dataset_long_img_features.csv", index=False)

## Collect weather data from different columns into a time series

In [49]:
def replace_nan(x):
    if x=="nan":
        return np.nan
    else :
        return float(x)

def convert_str_to_list(data, features):
    for feature in features : 
        data[feature]=data[feature].apply(lambda x: [ replace_nan(X) for X in x.strip('[]').split(",")])
    return data

In [50]:
def join_time_values(a, b, c, d):
    result = [0] * (len(a) + len(b) + len(c) + len(d))
    result[::4]  = a[::-1]
    result[1::4] = b[::-1]
    result[2::4] = c[::-1]
    result[3::4] = d[::-1]
    return result

In [51]:
temp = pd.read_csv('../data/temperature.csv', parse_dates=['date'])
temp = convert_str_to_list(temp, ['t_12', 't_6', 't_0', 't_18'])
#temp.head()

In [52]:
for row in (pbar := tqdm(temp.itertuples(), total=len(temp))):
    time_ser = join_time_values(row.t_0, row.t_6, row.t_12, row.t_18)
    temp.loc[temp['uid'] == row.uid, ['temp']] = str(time_ser)

temp = temp.drop(['t_12', 't_6', 't_0', 't_18'], axis=1)
temp.head()

100%|██████████| 23570/23570 [00:29<00:00, 786.69it/s]


,uid,latitude,longitude,date,split,temp
0,aabm,39.080319,-86.430867,2018-05-14,train,"[286.6784, 286.74725, 286.69934, 286.7568, 286..."
1,aabn,36.559700,-121.510000,2016-08-31,test,"[nan, nan, 286.99545, nan, nan, nan, 288.10834..."
2,aacd,35.875083,-78.878434,2020-11-19,train,"[290.66895, 286.5667, 285.7143, 291.8991, 285...."
3,aaee,35.487000,-79.062133,2016-08-24,train,"[301.63077, nan, nan, 314.93414, 301.9608, 297..."
4,aaff,38.049471,-99.827001,2019-07-23,train,"[306.29938, 298.71417, 300.36365, 315.71246, 3..."


In [53]:
# Write into new csv file
#temp.to_csv('../data/temperature_series.csv', index=False)

## Combine all features into complete dataset

In [54]:
img_features = pd.read_csv('../data/dataset_long_img_features.csv', parse_dates=['date'])
img_features.head()

,uid,latitude,longitude,date,split,region,severity,density,image_src,lake_size,mean_r,mean_g,mean_b,mean_ir,mean_h,mean_s,mean_v
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,NaN,2578.0,0.467581,0.607486,0.536248,0.424409,0.413782,0.234153,0.607841
1,aabn,36.559700,-121.510000,2016-08-31,test,NaN,NaN,NaN,no_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,NaN,176.0,0.112471,0.111118,0.057916,0.059888,0.163396,0.503357,0.115065
3,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,NaN,1495.0,0.065787,0.085121,0.044582,0.020998,0.245394,0.480389,0.085137


In [55]:
temp = pd.read_csv('../data/temperature_series.csv', parse_dates=['date'])
temp = convert_str_to_list(temp, ['temp'])
temp.head()


,uid,latitude,longitude,date,split,temp
0,aabm,39.080319,-86.430867,2018-05-14,train,"[286.6784, 286.74725, 286.69934, 286.7568, 286..."
1,aabn,36.559700,-121.510000,2016-08-31,test,"[nan, nan, 286.99545, nan, nan, nan, 288.10834..."
2,aacd,35.875083,-78.878434,2020-11-19,train,"[290.66895, 286.5667, 285.7143, 291.8991, 285...."
3,aaee,35.487000,-79.062133,2016-08-24,train,"[301.63077, nan, nan, 314.93414, 301.9608, 297..."
4,aaff,38.049471,-99.827001,2019-07-23,train,"[306.29938, 298.71417, 300.36365, 315.71246, 3..."


In [56]:
wind = pd.read_csv('../data/wind_series.csv', parse_dates=['date'])
wind = wind.drop(['longitude_trans', 'x_grid', 'y_grid'], axis=1)
wind = convert_str_to_list(wind, ['wind'])
wind.head()

,uid,latitude,longitude,date,split,wind
0,aabm,39.080319,-86.430867,2018-05-14,train,"[2.2833447, 1.94622, 1.331817, 1.2656376, 2.75..."
1,aabn,36.559700,-121.510000,2016-08-31,test,"[nan, nan, 3.9813075, nan, nan, nan, 3.7734618..."
2,aacd,35.875083,-78.878434,2020-11-19,train,"[2.005445, 3.5083356, 2.3487167, 2.2689152, 1...."
3,aaee,35.487000,-79.062133,2016-08-24,train,"[1.5693433, nan, nan, 6.309602, 1.4112688, 2.9..."
4,aaff,38.049471,-99.827001,2019-07-23,train,"[7.8561816, 6.940431, 7.351467, 10.175036, 9.9..."


In [57]:
elevation = pd.read_csv('../data/elevation_long.csv', parse_dates=['date'])
elevation.head()

,uid,latitude,longitude,date,split,region,severity,density,elev_mean,elev_median,elev_min,elev_max
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,0.058860,0.057391,0.057125,0.068623
1,aabn,36.559700,-121.510000,2016-08-31,test,NaN,NaN,NaN,0.028844,0.028692,0.028140,0.031199
2,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,0.044726,0.044747,0.039996,0.048706
3,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,0.050598,0.050579,0.043481,0.056447
4,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,0.174059,0.173836,0.171177,0.178763


In [58]:
radiation = pd.read_csv('../data/radiation_series.csv', parse_dates=['date'])
radiation = convert_str_to_list(radiation, ['rad_0_17_18'])
radiation = radiation.drop(['dswrf_17', 'dswrf_0', 'dswrf_18'], axis=1)
radiation = radiation.rename({'rad_0_17_18':'radiation'}, axis=1)
radiation.head()


,uid,latitude,longitude,date,split,radiation
0,aabm,39.080319,-86.430867,2018-05-14,train,"[0.0, 994.0, 948.0, 0.0, 391.0, 905.0, 0.0, 96..."
1,aabn,36.559700,-121.510000,2016-08-31,test,"[nan, 565.0, nan, nan, 781.0, 809.0, 265.6, 59..."
2,aacd,35.875083,-78.878434,2020-11-19,train,"[0.0, 380.8, 393.0, 0.0, 558.1, 492.2, 0.0, 11..."
3,aaee,35.487000,-79.062133,2016-08-24,train,"[0.0, 473.0, 875.0, 0.0, 938.0, 868.0, 0.0, 80..."
4,aaff,38.049471,-99.827001,2019-07-23,train,"[129.4, 975.0, 990.0, 128.7, 972.0, 987.0, 129..."


In [59]:
# list elevation columns for merging
elev_columns = ['uid', 'elev_mean','elev_median','elev_min','elev_max']
# merge all features into single dataframe
df_complete = img_features.merge(temp[['uid','temp']] , how="inner", on='uid' , validate="1:1")
df_complete = df_complete.merge(wind[['uid','wind']] , how="inner", on='uid' , validate="1:1")
df_complete = df_complete.merge(elevation[elev_columns] , how="inner", on='uid' , validate="1:1")
df_complete = df_complete.merge(radiation[['uid','radiation']] , how="inner", on='uid' , validate="1:1")


In [60]:
# save to csv file 
df_complete.to_csv('../data/dataset_long_all_feat_with_radiation.csv', index=False)

In [61]:
df = pd.read_csv('../data/dataset_long_all_feat_with_radiation.csv', parse_dates=['date'])

# use only data points after 2014-10-07 because of missing weather data
df = df[df['date'] > datetime.datetime(2014,10,7)]

df = convert_str_to_list(df, ['temp', 'wind', 'radiation'])
df.head()

,uid,latitude,longitude,date,split,region,severity,density,image_src,lake_size,...,mean_h,mean_s,mean_v,temp,wind,elev_mean,elev_median,elev_min,elev_max,radiation
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,NaN,2578.0,...,0.413782,0.234153,0.607841,"[286.6784, 286.74725, 286.69934, 286.7568, 286...","[2.2833447, 1.94622, 1.331817, 1.2656376, 2.75...",0.058860,0.057391,0.057125,0.068623,"[0.0, 994.0, 948.0, 0.0, 391.0, 905.0, 0.0, 96..."
1,aabn,36.559700,-121.510000,2016-08-31,test,NaN,NaN,NaN,no_image,NaN,...,NaN,NaN,NaN,"[nan, nan, 286.99545, nan, nan, nan, 288.10834...","[nan, nan, 3.9813075, nan, nan, nan, 3.7734618...",0.028844,0.028692,0.028140,0.031199,"[nan, 565.0, nan, nan, 781.0, 809.0, 265.6, 59..."
2,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,NaN,176.0,...,0.163396,0.503357,0.115065,"[290.66895, 286.5667, 285.7143, 291.8991, 285....","[2.005445, 3.5083356, 2.3487167, 2.2689152, 1....",0.044726,0.044747,0.039996,0.048706,"[0.0, 380.8, 393.0, 0.0, 558.1, 492.2, 0.0, 11..."
3,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,NaN,NaN,...,NaN,NaN,NaN,"[301.63077, nan, nan, 314.93414, 301.9608, 297...","[1.5693433, nan, nan, 6.309602, 1.4112688, 2.9...",0.050598,0.050579,0.043481,0.056447,"[0.0, 473.0, 875.0, 0.0, 938.0, 868.0, 0.0, 80..."
4,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,NaN,1495.0,...,0.245394,0.480389,0.085137,"[306.29938, 298.71417, 300.36365, 315.71246, 3...","[7.8561816, 6.940431, 7.351467, 10.175036, 9.9...",0.174059,0.173836,0.171177,0.178763,"[129.4, 975.0, 990.0, 128.7, 972.0, 987.0, 129..."
